In [1]:
import jax
jax.config.update("jax_enable_x64", True)

import jax.numpy as jnp
import numpy as np 
import s2scat 
import s2fft
import s2wav

In [2]:
L = 16
N = 3
reality = False
flm = jnp.array(np.random.randn(L, 2*L-1) + 1j*np.random.randn(L, 2*L-1))
filters = s2wav.filters.filters_directional_vectorised(L, N)[0]

In [4]:
precomps = s2scat.utility.kernels.generate_full_precompute(L,N)

In [4]:
coeffs_old = s2scat.core.scatter.scat_cov_dir(flm, 
    L, 
    N, 
    J_min=0,
    multiresolution=True, 
    for_synthesis=True, 
    filters=filters,
    precomps=precomps
)

In [5]:
coeffs_new = s2scat.core.scatter_new.directional(flm, 
    L, 
    N, 
    J_min=0,
    filters=filters,
    precomps=precomps
)

In [6]:
for i in range(6):
    np.testing.assert_allclose(coeffs_new[i], coeffs_old[i])

In [7]:
def old_func(flm):
    coeff = s2scat.core.scatter.scat_cov_dir(flm, 
        L, 
        N, 
        J_min=0,
        multiresolution=True, 
        for_synthesis=True, 
        filters=filters,
        precomps=precomps
    )
    loss = 0
    for i in range(6):
        loss += jnp.sum(jnp.abs(coeff[i]))
    return loss

def new_func(flm):
    coeff = s2scat.core.scatter_new.directional(flm, 
        L, 
        N, 
        J_min=0,
        filters=filters,
        precomps=precomps
    )
    loss = 0
    for i in range(6):
        loss += jnp.sum(jnp.abs(coeff[i]))
    return loss

